In [1]:
#Load txt file with information about the gage sites
import csv
gageSite = open("160921_GageSites_LA.txt","r")
csvReader = csv.reader(gageSite, delimiter=' ')

In [2]:
siteList = []

for line in csvReader:
    if line[-1]=="LA":
        siteID = line[5]
        place = line[-2][:-1]
        pl = line[-3]
        if pl != "near" and pl != "at" and pl != "nr" and pl != "of" and pl != "NR" and pl != "AT":
            place = ' '.join([pl, place])
        siteList.append([siteID,place])

In [3]:
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [4]:
#the following dataframe will be used later to identify the parish in which the gage is found
gages = pd.DataFrame(siteList,columns=['ID','City'])
gages.head()

,ID,City
0,02489500,Bogalusa
1,02491500,Franklinton
2,02492000,Bush
3,02492600,Pearl River
4,07344450,Greenwood


In [38]:
#Parsing the file with the daily gage height for all sites in Louisiana (LA)
gageMeanHeight = []
#gageHeight = open("Test.txt","r")
gageHeight = open("160921_DailyWaterData_LAbyCounty.txt","r")
csvR = csv.reader(gageHeight, delimiter="\t")
for line in csvR:
    if "#" not in line[0] and "agency_cd" in line[0]:
        siteNo_I = line.index("site_no")
        date_I = line.index("datetime")
        tmp = next((s for s in line if "00003" in s), None)
        #print tmp
        meanH_I = line.index(tmp)
        csvR.next()
    elif "#" not in line[0]:
        gageMeanHeight.append([line[siteNo_I], line[date_I], line[meanH_I]])
#print gageMeanHeight

gageMH = pd.DataFrame(gageMeanHeight,columns=['ID','Date','MeanHeight'])
gageMH.head()




,ID,Date,MeanHeight
0,02489500,1970-10-01,6.57
1,02489500,1970-10-02,6.46
2,02489500,1970-10-03,6.20
3,02489500,1970-10-04,5.97
4,02489500,1970-10-05,5.82


In [39]:
print len(gageMH.index)

1026350
